# Asynchronous Programming

## Key APIs

**`asyncio.run(coro())`**

Called from a regular function to drive a coroutine object that usually is the entry point for all the asynchronous code in the program.

**`asyncio.create_task(coro())`**

Called from a coroutine to schedule another coroutine to execute eventually.

**`await coro()`**
Called from a coroutine to transfer control to the coroutine object returned by `coro()`. This suspends the current coroutine until the body of `coro` returns.

## A Motivating Example

In [ ]:
import asyncio
import socket
from keyword import kwlist

MAX_KEYWORD_LEN = 4


async def probe(domain: str) -> tuple[str, bool]:
    loop = asyncio.get_running_loop()
    try:
        await loop.getaddrinfo(domain, None)
    except socket.gaierror:
        return (domain, False)
    return (domain, True)


async def main() -> None:
    names = (kw for kw in kwlist if len(kw) <= MAX_KEYWORD_LEN)
    domains = (f"{name}.dev".lower() for name in names)
    coros = [probe(domain) for domain in domains]
    for coro in asyncio.as_completed(coros):
        domain, found = await coro
        mark = "+" if found else " "
        print(f"{mark} {domain}")


if __name__ == "__main__":
    asyncio.run(main())

This is a common pattern for scripts that use asyncio: implement `main` as a coroutine, and drive it with `asyncio.run` inside the `if __name__ == '__main__':` block.

## Understanding the difference between coroutine objects and asyncio.Task objects

In [ ]:
import time
import asyncio


async def take_order(table):
    print(f"开始为 {table} 号桌点餐")
    await asyncio.sleep(1)
    print(f"{table} 号桌点餐完成")


async def main1():
    print("直接调用方式:")
    await take_order(1)  # 必须等待这个完成
    await take_order(2)  # 才能开始下一个
    await take_order(3)


async def main2():
    print("create_task 方式:")
    task1 = asyncio.create_task(take_order(1))  # 立即开始执行
    task2 = asyncio.create_task(take_order(2))  # 立即开始执行
    task3 = asyncio.create_task(take_order(3))  # 立即开始执行

    await task1  # 只是等待它们完成
    await task2
    await task3


if __name__ == "__main__":
    start = time.perf_counter()
    asyncio.run(main1())
    end = time.perf_counter()
    print(f"main1 总耗时: {end - start} 秒")

    start = time.perf_counter()
    asyncio.run(main2())
    end = time.perf_counter()
    print(f"main2 总耗时: {end - start} 秒")

## asyncio.gather

In [ ]:
import time
import asyncio


async def take_order(table):
    print(f"开始为 {table} 号桌点餐")
    await asyncio.sleep(1)
    print(f"{table} 号桌点餐完成")
    return f"订单 {table}"


async def main():
    orders = await asyncio.gather(take_order(1), take_order(2), take_order(3))
    return orders


if __name__ == "__main__":
    start = time.perf_counter()
    orders = asyncio.run(main())
    end = time.perf_counter()
    print(f"总耗时: {end - start} 秒")
    print(orders)

## asyncio.as_completed

In [ ]:
import time
import asyncio


async def take_order(table, time_to_wait):
    print(f"开始为 {table} 号桌点餐")
    await asyncio.sleep(time_to_wait)
    print(f"{table} 号桌点餐完成")
    return f"订单 {table}"


async def main():
    orders = []
    tasks = [take_order(1, 3), take_order(2, 2), take_order(3, 1)]

    for completed in asyncio.as_completed(tasks):
        order = await completed
        orders.append(order)
    return orders


if __name__ == "__main__":
    start = time.perf_counter()
    orders = asyncio.run(main())
    end = time.perf_counter()
    print(f"总耗时: {end - start} 秒")
    print(orders)